In [9]:
# These IPython-specific commands
# tell the notebook to reload imported
# files every time you rerun code. So
# you can modify inertial_wheel_pendulum.py
# and then rerun this cell to see the changes.
%load_ext autoreload
%autoreload 2

from inertial_wheel_pendulum import *
import math
import numpy as np
# Make numpy printing prettier
np.set_printoptions(precision=3, suppress=True)

# Define the upright fixed point here.
uf = np.array([0.])
xf = np.array([math.pi, 0, 0, 0])

# Pendulum params. You're free to play with these,
# of course, but we'll be expecting you to use the
# default values when answering questions, where
# varying these values might make a difference.
m1 = 1. # Default 1
l1 = 1. # Default 1
m2 = 2. # Default 2
l2 = 2. # Default 2
r = 1.0 # Default 1
g = 10  # Default 10
input_max = 10
pendulum_plant = InertialWheelPendulum(
    m1 = m1, l1 = l1, m2 = m2, l2 = l2, 
    r = r, g = g, input_max = input_max)

'''
Code submission for 3.1: 
Edit this method in `inertial_wheel_pendulum.py`
and ensure it produces reasonable A and B
'''
A, B = pendulum_plant.GetLinearizedDynamics(uf, xf)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def is_controllable(A, B):
    n = B.shape[0]
    m = B.shape[1]
    controllability_matrix = np.zeros((n, n*m))

    '''
    Code submission for 3.2: write code here
    to evaluate the controllability
    of A, B
    '''
    
    print(controllability_matrix)
    
    if A.shape[1] != B.shape[0]:
        return False
    
    terms = []
    for elt in range(0,n):
        term = np.dot(np.linalg.matrix_power(A,elt),B)
        controllability_matrix = np.insert(controllability_matrix, [elt], term, axis=1)

    print(controllability_matrix)
    
    controlRank = np.linalg.matrix_rank(controllability_matrix)    
    
    if controlRank == n:
        return True
    
    return False


In [11]:
from pydrake.all import (BasicVector, DiagramBuilder, FloatingBaseType,
                         LinearQuadraticRegulator, RigidBodyPlant,
                         RigidBodyTree, Simulator)

def create_reduced_lqr(A, B):
    '''
    Code submission for 3.3: Fill in the missing
    details of this function to produce a control
    matrix K, and cost-to-go matrix S, for the full
    (4-state) system.
    '''
  
    K = np.zeros((4))
    S = np.zeros((4, 4))
    
    # Q_reduced, A_reduced, and B_reduced
    # are all defined in the reduced state space
    # x = [theta_1, \dot{theta_1}, \dot{theta_2}]
    A_reduced = np.delete(np.delete(A, 1, 0), 1, 1)
    B_reduced = np.delete(B, 1, 0)
    Q_reduced = np.zeros((3, 3))
    R_reduced = np.zeros((1, 1))

    Q_reduced = np.diag([100., 10., 0.1])
    R_reduced[0] = [1.]
    
    K_reduced, S_reduced = LinearQuadraticRegulator(
        A_reduced, B_reduced, Q_reduced, R_reduced)

    K = np.insert(K_reduced, 1, 0., axis=1)
    S = np.insert(np.insert(S_reduced, 1, 0., axis=1), 1, 0., axis=0)
    
    return (K, S)
    
K, S = create_reduced_lqr(A, B)

In [28]:
from inertial_wheel_pendulum import *
from IPython.display import HTML
from inertial_wheel_pendulum_visualizer import *
import matplotlib.pyplot as plt

def badCalibration(x, offset):
    #poor calibration of the encoder on the base of the flywheel pendulum
    #in this case the state x is a set of angles + angular velocities
    #so offset should be in radians
    x[1] = x[1] + offset
    x_bad = x
    return x_bad

def underPowered(u0, offset):
    #underpowered base of pendulum actuator
    
    coeff = 1-offset
    u0 = coeff*u0
    return u0
    
def in_backlash_zone(x1, prevx1, rad_backlash, rad_tooth): 
    #t_i = ideal tooth thickness
    #t_a = actual tooth thickness
    #1 rotation of x_slices
    rot_sliced = (2*math.pi)/(rad_backlash+rad_tooth)
    
    # assume x= 0 starts on a tooth    

    backlash_zone = [] 
    
    start = 0
    while start < (2*math.pi):
        start = start + rad_tooth
        backlash_zone +=  [(start, start+rad_backlash)]
        start = start + rad_backlash
    
    if backlash_zone[-1][1] > (2*math.pi):
        backlash_zone = backlash_zone[:-1]
       
    
    if x1 > (2*math.pi):
        numRots = x1/(2*math.pi)
        x1 = x1 - floor(numRots)*(2*math.pi)
                
    
    for elt in backlash_zone: 
        if (x1 >= elt[0]) and (x1 <= elt[1]):
            return True
        
    return False
    
    
    
    
#try the above ones with the flywheel actuator?? 
#try using traj_opt 

def lqr_controller(x, offset_Rad, offset_Pwr, radbacklash, radtooth):
    # This should return a 1x1 u that is bounded
    # between -input_max and input_max.
    # Remember to wrap the angular values back to
    # [-pi, pi].
    
    u = np.zeros((1, 1))
    global xf, uf, K
    
 
    if radbacklash != 0:
        in_backlash_zone(x[0], radbacklash, radtooth)   
    if offset_Rad != 0:
        x = badCalibration(x, offset_Rad)
    
    

    x_lin = x - xf
    x_lin[0:2] = ( x_lin[0:2] + math.pi) % (2 * math.pi ) - math.pi
    u = -np.dot(K, x_lin)
    u[0] = max(-input_max, min(input_max, u[0]))
    
    if offset_Pwr != 0:
        u[0] = underPowered(u[0], offset_Pwr)
    
    return u


In [29]:
import matplotlib.pyplot as plt
import time



# Calculates the closed loop f(x) at xn
def calcF(xn):
    # Feel free to bring in whatever
    # global variables you need, e.g.:
    global pendulum_plant
    
    '''
    Code submission for 3.5: populate
    this function to calculate the closed-loop
    system dynamics f(x) at the input point.
    
    '''
    u = lqr_controller(xn, offsetRad, offsetPwr, rad_backlash, rad_tooth)
    return pendulum_plant.evaluate_f(u, xn)

# Calculates V(xn)
def calcV(xn):
    # Feel free to bring in whatever
    # global variables you need

    '''
    Code submission for 3.5: populate
    this function to calculate V(x)
    at the input point.
    '''
    global S, xf
    x_err = xn - xf
    x_err[0:2] = ( x_err[0:2] + math.pi) % (2 * math.pi ) - math.pi
    return np.dot(x_err, np.dot(S, x_err))

# Calculates \dot{V}(xn).
def calcVdot(xn):
    # Feel free to bring in whatever
    # global variables you need
    
    '''
    Code submission for 3.5: populate
    this function to calculate Vdot(x)
    at the input point.
    '''
    global xf
    
    x_err = xn - xf
    x_err[0:2] = ( x_err[0:2] + math.pi) % (2 * math.pi ) - math.pi
    # It's important to still calculate F using xn, not
    # the error coordinate.
    return np.dot(np.dot(S, x_err), calcF(xn))


def calc_over_array(f):
    return np.array([[[f([dx, 0., dy, dz]) for dz in z] for dx in x] for dy in y])



In [ ]:
def estimate_rho(V, Vdot):
    '''
    Code submission for 3.6
    Fill in this function to use the samples of V and Vdot
    (Each array has dimension [n_bins, n_bins, n_bins_theta2d])
    to compute a maximal rho indicating the region of attraction
    of the fixed point at the upright.
    '''
    
    smallest_counterexample_V = np.max(V)
    for dx in range(V.shape[0]):
        for dy in range(V.shape[1]):
            for dz in range(V.shape[2]):
                if Vdot[dx, dy, dz] >= 0 and V[dx, dy, dz] < smallest_counterexample_V:
                    if V[dx, dy, dz] != 0:
                        smallest_counterexample_V = V[dx, dy, dz]
    # If we take rho slightly smaller, we protect ourselves against
    # discretization problems (maybe we didn't sample exactly at the true rho,
    # so we may have overestimated). It's likely that all V lower than this number
    # have negative Vdot, so we've found the ROA.
    return smallest_counterexample_V - 0.01
    
    return 0.

def ROA_area(Vsamples, Vdotsamples):

    rho = estimate_rho(Vsamples, Vdotsamples)
    #print "Region of attraction estimated at V(x) <= ", rho

    # Plot Vdot again, but overlay the region of attraction -- which,
    # for quadratic V, is an ellipse.
    #fig = plt.figure()
    #ax = fig.add_subplot(1, 1, 1)
    #fig.set_size_inches(12,6)
    #plt.pcolormesh(Xplot, Yplot, Vdot_viz, vmin=0, vmax=1.0)

    # The part of S we care about is the 2x2 submatrix from the 1st and 3rd rows
    # and columns.
    S_sub = np.reshape(S[[0, 2, 0, 2], [0, 0, 2, 2]], (2, 2))
    # Extract its eigenvalues and eigenvectors, which tell us
    # the axes of the ellipse
    ellipseInfo = np.linalg.eig(S_sub)
    # Eigenvalues are 1/r^2, Eigenvectors are axis directions
    axis_1 = ellipseInfo[1][0, :]
    if ellipseInfo[0][0] > 0 and ellipseInfo[0][1] > 0:
        r1 = math.sqrt(rho)/math.sqrt(ellipseInfo[0][0])
        axis_2 = ellipseInfo[1][1, :]
        r2 = math.sqrt(rho)/math.sqrt(ellipseInfo[0][1])
        angle = math.atan2(-axis_1[1], axis_1[0])
        from matplotlib.patches import Ellipse
        

        # Report an interesting number that is easy to compute
        # from the ellipse info
        
        roa_area = math.pi*r1*r2
        
        return [rho, roa_area]
    else:
        return [rho, "S_sub had nonpositive eigenvalues. That shouldn't happen."]

#plt.title("Vdot(x) overlaid with estimated ROA at dtheta_2 = %f" % z[theta2d_plotting_slice])
#plt.xlabel("theta_1")
#plt.ylabel("dtheta_1");


offsetRad = 0
offsetPwr = 0
rad_backlash = 0.01
rad_tooth = 0.05
prevx1 = 3


while offsetRad <= 6.28:  
    print offsetRad
    start_time = time.time()

    # Sample f, V, and Vdot over
    # a grid defined by these parameters.
    # (Odd numbers are good because there'll be
    # a bin at exactly the origin.
    # These are slightly strange numbers as we've
    # tried to default these to something as small
    # as possible while still giving reasonable results.
    # Feel free to increase if your computer and patience
    # can handle it.)
    n_bins = 41
    n_bins_theta2d = 7
    # For theta and thetad, we only need to span
    # a small region around the fixed point
    theta_width = 4
    thetad_width = 4
    # For \dot{theta_2}, though, the default
    # parameters for our pendulum lead us to
    # need to search larger absolute \dot{theta_2}
    # values (because the inertial wheel is relatively
    # light).
    theta2d_width = 50
    # Do the actual sampling....
    x = np.linspace(xf[0]-theta_width, xf[0]+theta_width, n_bins)
    y = np.linspace(xf[2]-thetad_width, xf[2]+thetad_width, n_bins)
    z = np.linspace(xf[3]-theta2d_width, xf[3]+theta2d_width, n_bins_theta2d)
    X, Y, Z = np.meshgrid(x, y, z, indexing="ij")

    V_samples = calc_over_array(calcV)
    f_samples = calc_over_array(calcF)
    Vdot_samples = calc_over_array(calcVdot)

    elapsed = time.time() - start_time
    #print "Computed %d x %d x %d sampling in %f seconds" % (n_bins, n_bins, n_bins_theta2d, elapsed)

    roaArea = ROA_area(V_samples,  Vdot_samples)
    offsetRad += 0.5

    
    print roaArea[1]


In [32]:
##Visualize everything if you want##


# Run forward simulation from the specified initial condition
duration = 20.
x0 = [3, 0.0, 0.0, 5.0]

input_log, state_log = \
    RunSimulation(pendulum_plant,
              lqr_controller,
              x0=x0,
              duration=duration)

    
# Visualize state and input traces
fig = plt.figure().set_size_inches(6, 6)
for i in range(4):
    plt.subplot(5, 1, i+1)
    plt.plot(state_log.sample_times(), state_log.data()[i, :])
    plt.grid(True)
    plt.ylabel("x[%d]" % i)
plt.subplot(5, 1, 5)
plt.plot(input_log.sample_times(), input_log.data()[0, :])
plt.ylabel("u[0]")
plt.xlabel("t")
plt.grid(True)

# Visualize the simulation
viz = InertialWheelPendulumVisualizer(pendulum_plant)
ani = viz.animate(input_log, state_log, 30, repeat=True)
plt.close(viz.fig)
HTML(ani.to_html5_video())

TypeError: lqr_controller() takes exactly 5 arguments (1 given)